단방향 LSTM으로 텍스트 분류를 수행할 수도 있으나, 양방향 LSTM을 사용하는 것이 더 강력함. 여기에 추가적으로 어텐션 메커니즘을 사용할 수도 있음

### 1. IMDB 리뷰 데이터 전처리

In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

IMDB 리뷰 데이터는 최대 단어 개수를 10,000으로 제한하고 훈련데이터와 테스트 데이터를 받아옴

In [2]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

17465344/17464789 [==============================] - 0s 0us/step


In [4]:
print(X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [5]:
X_train.shape, X_test.shape

((25000,), (25000,))

In [6]:
# 데이터가 이미 정수 인코딩된 상태이므로, 남은 전처리는 패딩 뿐임. 리뷰의 최대 길이와 평균길이 확인

print('리뷰의 최대 길이: ', max(len(x) for x in X_train))
print('리뷰의 평균 길이: ', sum(len(x) for x in X_train) / len(X_train))

리뷰의 최대 길이:  2494
리뷰의 평균 길이:  238.71364


리뷰의 최대길이는 2,494이며, 리뷰의 평균길이는 약 238임. 여기에서는 평균 길이보다 좀더 크게 데이터를 패딩함

In [7]:
max_length = 500
X_train = pad_sequences(X_train, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

훈련용 리뷰와 테스트용 리뷰의 길이 모두 500이 됨

### 2. 바다나우 어텐션(Bahdanau Attention)
------
여기서 사용할 어텐션은 바다나우 어텐션임

어텐션 스코어 함수란, 주어진 query와 모든 key에 대해 유사도를 측정하는 함수를 말함. 그리고 닷 프로덕트 어텐션에서는 query와 key의 유사도를 구하는 방법이 내적(dot product)이었음. 다음은 닷 프로덕트 어텐션의 스코어 함수를 보여줌

$$score(query,\ key) = query^Tkey$$

바다나우 어텐션은 아래와 같이 어텐션 스코어 함수를 사용함

$$score(query,\ key) = V^Ttanh(W_{1}key + W_{2}query)$$


이 어텐션 스코어 함수를 사용하여 어텐션 메커니즘을 구현하면 됨. 그런데 텍스트 분류에서 어텐션 메커니즘을 사용하는 이유는 무엇을까? RNN의 마지막 은닉상태는 예측을 위해 사용됨. 그런데 이 RNN의 마지막 은닉상태는 몇가지 유용한 정보들을 손실한 상태임. 그래서 RNN이 time step을 지나며 손실했던 정보를 다시 참고하고자 하는 것임

이는 다시 말해 RNN의 모든 은닉상태들을 다시 한번 참고하겠다는 것이며, 이를 위해 어텐션 메커니즘을 사용하는 것임

In [8]:
import tensorflow as tf

In [9]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)  # value
        
    def call(self, values, query): # 단, key와 value는 같음
        # query shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # score 계산을 위해 뒤에서 할 덧셈을 위해 차원을 변경해 줌
        hidden_with_time_axis = tf.expand_dims(query, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        return context_vector, attention_weights

### 3. 양방향 LSTM + 어텐션 메커니즘(BiLSTM with Attention Mechanism)

In [10]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

##### 1) 입력층과 임베딩층 설계

In [12]:
max_length, vocab_size

(500, 10000)

In [13]:
sequece_input = Input(shape=(max_length,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_length, \
                               mask_zero=True)(sequece_input)

10000개 단어들을 128차원의 벡터로 임베딩하도록 설계함

In [14]:
embedded_sequences.shape

TensorShape([None, 500, 128])

##### 2) 양방향 LSTM 설계  
* 두층을 설계하기 때문에, 첫번째 층에 두번째 층을 쌓을 예정이므로 return_sequences=True로 설정해야 함

In [15]:
# 첫번째 층
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences=True))(embedded_sequences)

In [16]:
# 두번째 층 설계
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional\
(LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

각 상태의 크기를 출력해 봄

In [17]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


순방향 LSTM의 은닉상태와 셀상태를 forward_h, forward_c에 저장하고, 역방향 LSTM의 은닉상태와 셀상태를 backward_h, backward_c에 저장함

각 은닉상태나 셀상태의 경우, 128차원을 가지고, lstm은 (500 x 128)의 크기를 가짐. forward 방향과 backward 방향이 연결된 hidden state 벡터가 모든 시점에 대해 존재함을 의미함

양방향 LSTM을 사용할 경우에는 순방향 LSTM과 역방향 LSTM 각각 은닉상태와 셀상태를 가지므로, 양방향 LSTM의 은닉상태와 셀상태를 사용하려면 두방향의 LSTM의 상태들을 연결해주면 됨

In [18]:
state_h = Concatenate()([forward_h, backward_h])  # 은닉상태
state_c = Concatenate()([forward_c, backward_c])  # 셀 상태

어텐션 메커니즘에서는 은닉상태를 사용함. 이를 입력으로 컨텍스트 벡터를 얻음

In [19]:
attention = BahdanauAttention(64) #  가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

In [20]:
context_vector.shape, attention_weights.shape

(TensorShape([None, 128]), TensorShape([None, 500, 1]))

컨텍스트 벡터를 밀집층(dense layer)에 통과시키고, 이진분류이므로 최종 출력층에 1개의 뉴런을 배치하고, 활성화 함수로 시그모이드 함수를 사용함

In [21]:
dense1 = Dense(20, activation='relu')(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation='sigmoid')(dropout)
model = Model(inputs=sequece_input, outputs=output)

옵티마이저로 아담 옵티마이저를 사용하고 모델을 컴파일함

In [22]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

시그모이드 함수를 사용하므로 손실함수로 binary_crossentropy를 사용함. 다음으로 모델 훈련함

In [23]:
history = model.fit(X_train, y_train, epochs=3, batch_size=256, 
                    validation_data=(X_test, y_test), verbose=1)

Epoch 1/3
98/98 [==============================] - 2890s 29s/step - loss: 0.4727 - accuracy: 0.7652 - val_loss: 0.2941 - val_accuracy: 0.8791
Epoch 2/3
98/98 [==============================] - 2833s 29s/step - loss: 0.2439 - accuracy: 0.9131 - val_loss: 0.2887 - val_accuracy: 0.8830
Epoch 3/3
98/98 [==============================] - 2855s 29s/step - loss: 0.1874 - accuracy: 0.9350 - val_loss: 0.3326 - val_accuracy: 0.8772


검증 데이터로 테스트 데이터를 사용하여 에포크가 끌날때마다 테스트 데이터에 대한 정확도를 출력하도록 함

In [25]:
print('\n 테스트 정확도: %.4f' % (model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 137s 175ms/step - loss: 0.3326 - accuracy: 0.8772

 테스트 정확도: 0.8772


87.72% 정확도를 얻음